*This example is adapted from Continual AI Avalanche quick start https://avalanche.continualai.org/*

# Incremental Learning with Ray AIR

In this example, we show how to use Ray AIR to incrementally train a simple image classification PyTorch model
on a stream of incoming tasks.

Each task is a random permutation of the MNIST Dataset, which is a common benchmark
used for continual training. After training on all the
tasks, the model is expected to be able to make predictions on data from any task.

In this example, we use just a naive finetuning strategy, where the model is trained
on each task, without any special methods to prevent [catastrophic forgetting](
https://en.wikipedia.org/wiki/Catastrophic_interference). Model performance is
expected to be poor.

More precisely, this example showcases domain incremental training, in which during
prediction/testing
time, the model is asked to predict on data from tasks trained on so far with the
task ID not provided. This is opposed to task incremental training, where the task ID is
provided during prediction/testing time.

For more information on the 3 different categories for incremental/continual
learning, please see ["Three scenarios for continual learning" by van de Ven and Tolias](https://arxiv.org/pdf/1904.07734.pdf)

This example will cover the following:
1. Loading a PyTorch Dataset to Ray Datasets
2. Create an `Iterator[ray.data.Datasets]` abstraction to represent a stream of data to train on for incremental training.
3. Implement a custom Ray AIR preprocessor to preprocess the Dataset.
4. Incrementally train a model using data parallel training.
5. Use our trained model to perform batch prediction on test data.
6. Incrementally deploying our trained model with Ray Serve and performing online prediction queries.

# Step 1: Installations and Initializing Ray

To get started, let's first install the necessary packages: Ray AIR, torch, and torchvision. Uncomment the below lines and run the cell to install the necessary packages.

In [1]:
# !pip install -q "ray[air]"
# !pip install -q torch
# !pip install -q torchvision

Then, let's initialize Ray! We can just import and call `ray.init()`. If you are running on a Ray cluster, then you can do `ray.init("auto")` to connect to the cluster instead of initiailzing a new local Ray instance.

In [2]:
import ray
ray.init()
# If runnning on a cluster, use the below line instead.
# ray.init("auto")

2022-07-20 21:23:11,138	INFO services.py:1483 -- View the Ray dashboard at http://127.0.0.1:8265


Python version:,3.7.10
Ray version:,3.0.0.dev0
Dashboard:,http://127.0.0.1:8265


# Step 2: Define our PyTorch Model

Now that we have the necessary installations, let's define our PyTorch model. For this example to classify MNIST images, we will use a simple multi-layer perceptron.

In [3]:
import torch.nn as nn

class SimpleMLP(nn.Module):
    def __init__(self, num_classes=10, input_size=28 * 28):
        super(SimpleMLP, self).__init__()

        self.features = nn.Sequential(
          nn.Linear(input_size, 512),
          nn.ReLU(inplace=True),
          nn.Dropout(),
        )
        self.classifier = nn.Linear(512, num_classes)
        self._input_size = input_size

    def forward(self, x):
        x = x.contiguous()
        x = x.view(-1, self._input_size)
        x = self.features(x)
        x = self.classifier(x)
        return x

/Users/jiaodong/anaconda3/envs/ray3.7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step 3: Create the Stream of tasks

We can now create a stream of tasks (where each task contains a dataset to train on). For this example, we will create an artificial stream of tasks consisting of
permuted variations of MNIST, which is a classic benchmark in continual learning
research.

For real-world scenarios, this step is not necessary as fresh data will already be
arriving as a stream of tasks. It does not need to be artificially created.

## 3a: Load MNIST Dataset to a Ray Dataset

Let's first define a simple function that will return the original MNIST Dataset as a distributed Ray Dataset. Ray Datasets are the standard way to load and exchange data in Ray libraries and applications, read more about them [here](https://docs.ray.io/en/latest/data/dataset.html)!

The function in the below code snippet does the following:
1. Downloads the MNIST Dataset from torchvision in-memory
2. Loads the in-memory Torch Dataset into a Ray Dataset
3. Converts the Ray Dataset into a Pandas format. Instead of the Ray Dataset iterating over tuples, it will have 2 columns: "image" & "label". 
<!-- TODO: Figure out when and how to use TensorArray extension -->
<!-- The image will be stored as a multi-dimensional tensor (via the [TensorArray format](https://docs.ray.io/en/latest/data/dataset-tensor-support.html) instead of a PIL image).  -->
This will allow us to apply built-in preprocessors to the Ray Dataset and allow Ray Datasets to be used with Ray AIR Predictors.
 <!-- and also means that any transformations done to the images can be done in a zero-copy fashion. -->

For this example, since we are just working with MNIST dataset, which is small, we use the [`SimpleTorchDataSource`](https://docs.ray.io/en/master/data/package-ref.html?highlight=SimpleTorchDatasource#ray.data.datasource.SimpleTorchDatasource) which just loads the full MNIST dataset into memory.

For loading larger datasets in a parallel fashion, you should use [Ray Dataset's additional read APIs](https://docs.ray.io/en/master/data/dataset.html#supported-input-formats) to load data from parquet, csv, image files, and more!

In [4]:
import pandas as pd

import torchvision
from torchvision.transforms import RandomCrop

import ray
from ray.data.datasource.torch_datasource import SimpleTorchDatasource


def get_mnist_dataset(train: bool = True) -> ray.data.Dataset:
    """Returns MNIST Dataset as a ray.data.Dataset.
    
    Args:
        train: Whether to return the train dataset or test dataset.
    """

    def mnist_dataset_factory():
        if train:
            # Only perform random cropping on the Train dataset.
            transform = RandomCrop(28, padding=4)
        else:
            transform = None
        return torchvision.datasets.MNIST("./data", download=True, train=train, transform=transform)

    def convert_batch_to_pandas(batch):
        images = [np.array(item[0]) for item in batch]
        labels = [item[1] for item in batch]

        df = pd.DataFrame({"image": images, "label": labels})

        return df

    mnist_dataset = ray.data.read_datasource(
        SimpleTorchDatasource(), dataset_factory=mnist_dataset_factory
    )
    mnist_dataset = mnist_dataset.map_batches(convert_batch_to_pandas)
    return mnist_dataset

## 3b: Create our Stream abstraction

Now we can create our "stream" abstraction. This abstraction provides two
methods (`generate_train_stream` and `generate_test_stream`) that each returns an Iterator
over Ray Datasets. Each item in this iterator contains a unique permutation of
MNIST, and is one task that we want to train on.

In this example, "the stream of tasks" is contrived since all the data for all tasks exist already in an offline setting. For true online continual learning, you would want to implement a custom dataset iterator that reads from some stream datasource to produce new tasks. The only abstraction that's needed is `Iterator[ray.data.Dataset]`.

Note that the test dataset stream has the same permutations that are used for the training dataset stream. In general for continual learning, it is expected that the data distribution of the test/prediction data follows what the model was trained on. If you notice that the distribution of new prediction queries is changing compared to the distribution of the training data, then you should probably trigger training of a new task.

In [5]:
from typing import Iterator, List
import random
import numpy as np

from ray.data import ActorPoolStrategy


class PermutedMNISTStream:
    """Generates streams of permuted MNIST Datasets.
    
    Example:
        
        permuted_mnist = PermutedMNISTStream(n_tasks=3)
        train_stream = permuted_mnist.generate_train_stream()
        
        # Iterate through the train_stream
        for train_dataset in train_stream:
            ...
    
    Args:
        n_tasks: The number of tasks to generate.
    """

    def __init__(self, n_tasks: int = 3):
        self.n_tasks = n_tasks
        self.permutations = [
            np.random.permutation(28 * 28) for _ in range(self.n_tasks)
        ]

        self.train_mnist_dataset = get_mnist_dataset(train=True)
        self.test_mnist_dataset = get_mnist_dataset(train=False)

    def random_permute_dataset(
        self, dataset: ray.data.Dataset, permutation: np.ndarray
    ):
        """Randomly permutes the pixels for each image in the dataset."""

        class PixelsPermutation(object):
            def __call__(self, batch):
                batch["image"] = batch["image"].map(lambda image: image.reshape(-1)[permutation].reshape(28, 28))
                return batch

        return dataset.map_batches(PixelsPermutation, compute=ActorPoolStrategy(), batch_format="pandas")

    def generate_train_stream(self) -> Iterator[ray.data.Dataset]:
        for permutation in self.permutations:
            permuted_mnist_dataset = self.random_permute_dataset(
                self.train_mnist_dataset, permutation
            )
            yield permuted_mnist_dataset

    def generate_test_stream(self) -> Iterator[ray.data.Dataset]:
        for permutation in self.permutations:
            mnist_dataset = get_mnist_dataset(train=False)
            permuted_mnist_dataset = self.random_permute_dataset(
                self.test_mnist_dataset, permutation
            )
            yield permuted_mnist_dataset

    def generate_test_samples(self, num_samples: int = 10) -> List[np.ndarray]:
      """Generates num_samples permuted MNIST images."""
      random_permutation =  random.choice(self.permutations)
      return self.random_permute_dataset(self.test_mnist_dataset.random_shuffle().limit(num_samples), random_permutation).to_pandas()["image"].to_list()



# Step 4: Define the logic for Training and Inference/Prediction

Now that we can get an Iterator over Ray Datasets, we can incrementally train our model in a data parallel fashion via Ray Train, while incrementally deploying our model via Ray Serve. Let's define some helper functions to allow us to do this!

If you are not familiar with data parallel training, it is a form of distributed training strategies, where we have multiple model replicas, and each replica trains on a different batch of data. After each batch, the gradients are synchronized across the replicas. This effecitively allows us to train on more data in a shorter amount of time.

## 4a: Define our training logic for each Data Parallel worker

The first thing we need to do is to define the training loop that will be run on each training worker. 

The training loop takes in a `config` Dict as an argument that we can use to pass in any configurations for training.

This is just standard PyTorch training, with the difference being that we can leverage [Ray Train's utility functions](https://docs.ray.io/en/master/train/api.html#training-function-utilities) and [Ray AIR Sesssion](https://docs.ray.io/en/master/ray-air/package-ref.html#module-ray.air.session):
- `ray.train.torch.prepare_model(...)`: This will prepare the model for distributed training by wrapping it in PyTorch `DistributedDataParallel` and moving it to the correct accelerator device.
- `ray.air.session.get_dataset_shard(...)`: This will get the Ray Dataset shard for this particular Data Parallel worker.
- `ray.air.session.report({}, checkpoint=...)`: This will tell Ray Train to persist the provided `Checkpoint` object.
- `ray.air.session.get_checkpoint()`: Returns a checkpoint to resume from. This is useful for either fault tolerance purposes, or for our purposes, to continue training the same model on a new incoming dataset.

In [6]:
from ray import train
from ray.air import session, Checkpoint

from torch.optim import SGD
from torch.nn import CrossEntropyLoss

from torch.nn.modules.utils import consume_prefix_in_state_dict_if_present

def train_loop_per_worker(config: dict):
    num_epochs = config["num_epochs"]
    learning_rate = config["learning_rate"]
    momentum = config["momentum"]
    batch_size = config["batch_size"]

    model = SimpleMLP(num_classes=10)

    # Load model from checkpoint if there is a checkpoint to load from.
    checkpoint_to_load = session.get_checkpoint()
    if checkpoint_to_load:
        state_dict_to_resume_from = checkpoint_to_load.to_dict()["model"]
        model.load_state_dict(state_dict=state_dict_to_resume_from)

    model = train.torch.prepare_model(model)

    optimizer = SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    criterion = CrossEntropyLoss()

    # Get the Ray Dataset shard for this data parallel worker, and convert it to a PyTorch Dataset.
    dataset_shard = session.get_dataset_shard("train").to_torch(
        label_column="label",
        batch_size=batch_size,
        unsqueeze_feature_tensors=False,
        unsqueeze_label_tensor=False,
    )

    for epoch_idx in range(num_epochs):
        running_loss = 0
        for iteration, (train_mb_x, train_mb_y) in enumerate(dataset_shard):
            optimizer.zero_grad()
            train_mb_x = train_mb_x.to(train.torch.get_device())
            train_mb_y = train_mb_y.to(train.torch.get_device())

            # Forward
            logits = model(train_mb_x)
            # Loss
            loss = criterion(logits, train_mb_y)
            # Backward
            loss.backward()
            # Update
            optimizer.step()

            running_loss += loss.item()
            if session.get_world_rank() == 0 and iteration % 500 == 0:
                print(f"loss: {loss.item():>7f}, epoch: {epoch_idx}, iteration: {iteration}")

        # Checkpoint model after every epoch.
        state_dict = model.state_dict()
        consume_prefix_in_state_dict_if_present(state_dict, "module.")
        checkpoint = Checkpoint.from_dict(dict(model=state_dict))
        session.report({"loss": running_loss}, checkpoint=checkpoint)

## 4b: Define our Preprocessor

Next, we define our `Preprocessor` to preprocess our data before training and prediction. Our preprocessor will normalize the MNIST Images by the mean and standard deviation of the MNIST training dataset. This is a common operation to do on MNIST to improve training: https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457

In [7]:
from ray.data.preprocessors import BatchMapper
from ray.data.extensions import TensorArray

from torchvision import transforms

def preprocess_images(df: pd.DataFrame) -> pd.DataFrame:
    """Preprocess images by scaling each channel in the image."""

    torchvision_transforms = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )

    df["image"] = TensorArray([torchvision_transforms(image) for image in df["image"]])
    return df

mnist_normalize_preprocessor = BatchMapper(fn=preprocess_images)

## 4c: Define logic for Batch/Offline Prediction.

After training on each task, we want to use our trained model to do batch (i.e. offline) inference on a test dataset. 

To do this, we leverage the built-in `ray.air.BatchPredictor`. We define a `batch_predict` function that will take in a Checkpoint and a Test Dataset and outputs the accuracy our model achieves on the test dataset.

In [8]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.torch import TorchPredictor

def batch_predict(checkpoint: ray.air.Checkpoint, test_dataset: ray.data.Dataset) -> float:
  """Perform batch prediction on the provided test dataset, and return accuracy results."""

  batch_predictor = BatchPredictor.from_checkpoint(checkpoint, predictor_cls=TorchPredictor, model=SimpleMLP(num_classes=10))
  model_output = batch_predictor.predict(
            data=test_dataset, feature_columns=["image"], keep_columns=["label"]
        )
  
  # Postprocess model outputs.
  # Convert logits outputted from model into actual class predictions.
  def convert_logits_to_classes(df):
     best_class = df["predictions"].map(lambda x: np.array(x).argmax())
     df["predictions"] = best_class
     return df
    
  prediction_results = model_output.map_batches(convert_logits_to_classes, batch_format="pandas")
  
  # Then, for each prediction output, see if it matches with the ground truth
  # label.
  def calculate_prediction_scores(df):
      return pd.DataFrame({"correct": df["predictions"] == df["label"]})

  correct_dataset = prediction_results.map_batches(
      calculate_prediction_scores, batch_format="pandas"
  )

  return correct_dataset.sum(on="correct") / correct_dataset.count()

## 4d: Define logic for Deploying and Querying our model

In addition to batch inference, we also want to deploy our model so that we can submit live queries to it for online inference. We use Ray Serve's `PredictorDeployment` utility to deploy our trained model. 

Once we deploy the model, we can send HTTP requests to our deployment.

In [9]:
from typing import List
import requests
from requests import Response
import numpy as np

from ray.serve.http_adapters import NdArray


def deploy_model(checkpoint: ray.air.Checkpoint) -> str:
  """Deploys the model from the provided Checkpoint and returns the URL for the endpoint of the model deployment."""
  def json_to_pandas(payload: NdArray) -> pd.DataFrame:
      """Accepts an NdArray JSON from an HTTP body and converts it to a Pandas dataframe."""
      # Have to explicitly convert to float since np.array reads as a double.
      arr = np.array(payload.array, dtype=np.float32)
      # We have to specify an image column since our preprocessor requires it.
      df = pd.DataFrame({"image": [arr]})
      return df

  deployment = PredictorDeployment.options(name="mnist_model", route_prefix="/mnist_predict", version=f"v{task_idx}", num_replicas=2)
  deployment.deploy(
    batching_params=dict(max_batch_size=10, batch_wait_timeout_s=5),
    http_adapter=json_to_pandas, 
    predictor_cls=TorchPredictor, 
    checkpoint=latest_checkpoint, 
    model=SimpleMLP(num_classes=10)
  )
  return deployment.url

# Function that queries our deployed model
def query_deployment(test_samples: List[np.ndarray], endpoint_uri: str) -> List[Response]:
  """Given a set of test samples, queries the model deployment at the provided endpoint and returns the results."""
  results = []
  # Have to convert to Python List since Numpy arrays are not Json serializable.
  for sample in test_samples:
    results.append(requests.post(endpoint_uri, json={"array": sample.tolist()}))
  # TODO: Figure out how Serve deals with Pandas DataFrame returned by Predictors.
  return results

# Step 5: Putting it all together

Once we have defined our training logic and our preprocessor, we can put everything together!

For each dataset in our stream, we do the following:
1. Train on the dataset in Data Parallel fashion. We create a `TorchTrainer`, specify the config for the training loop we defined above, the dataset to train on, and how much we want to scale. `TorchTrainer` also accepts a `checkpoint` arg to continue training from a previously saved checkpoint.
2. Get the saved checkpoint from the training run.
3. Test our trained model on a test set containing test data from all the tasks trained on so far.
3. After training on each task, we deploy our model so we can query it for predictions.

In this example, the training and test data for each task is well-defined beforehand by the benchmark. For real-world scenarios, this probably will not be the case. It is very likely that the prediction requests after training on one task will become the training data for the next task. 


In [10]:
from ray.train.torch import TorchTrainer
from ray.air.config import ScalingConfig
from ray.train.torch import TorchPredictor
from ray import serve
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import json_to_ndarray

# The number of tasks (i.e. datasets in our stream) that we want to use for this example.
n_tasks = 3

# Number of epochs to train each task for.
num_epochs = 4
# Batch size.
batch_size = 32
# Optimizer args.
learning_rate = 0.001
momentum = 0.9

# Number of data parallel workers to use for training.
num_workers = 1
# Whether to use GPU or not.
use_gpu = False

permuted_mnist = PermutedMNISTStream(n_tasks=n_tasks)
train_stream = permuted_mnist.generate_train_stream()
test_stream = permuted_mnist.generate_test_stream()

latest_checkpoint = None

accuracy_for_all_tasks = []
task_idx = 0
all_test_datasets_seen_so_far = []
for train_dataset, test_dataset in zip(train_stream, test_stream):
  print(f"Starting training for task: {task_idx}")
  task_idx += 1

  # *********Training*****************

  trainer = TorchTrainer(
          train_loop_per_worker=train_loop_per_worker,
          train_loop_config={
              "num_epochs": num_epochs,
              "learning_rate": learning_rate,
              "momentum": momentum,
              "batch_size": batch_size,
          },
          # Have to specify trainer_resources as 0 so that the example works on Colab. 
          scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu, trainer_resources={"CPU": 0}),
          datasets={"train": train_dataset},
          preprocessor=BatchMapper(fn=preprocess_images),
          resume_from_checkpoint=latest_checkpoint,
      )
  result = trainer.fit()
  latest_checkpoint = result.checkpoint

  # **************Batch Prediction**************************

  # We can do batch prediction on the test data for the tasks seen so far.
  # TODO: Fix type signature in Ray Datasets
  # TODO: Fix dataset.union when used with empty list.
  if len(all_test_datasets_seen_so_far) > 0:
    full_test_dataset = test_dataset.union(*all_test_datasets_seen_so_far)
  else:
    full_test_dataset = test_dataset

  all_test_datasets_seen_so_far.append(test_dataset)

  accuracy_for_this_task = batch_predict(latest_checkpoint, full_test_dataset)
  print(f"Accuracy for task {task_idx}: {accuracy_for_this_task}")
  accuracy_for_all_tasks.append(accuracy_for_this_task)

  # *************Model Deployment & Online Inference***************************
  
  # We can also deploy our model to do online inference with Ray Serve.
  # Start Ray Serve.
  serve.start()
  test_samples = permuted_mnist.generate_test_samples()
  endpoint_uri = deploy_model(latest_checkpoint)
  online_inference_results = query_deployment(test_samples, endpoint_uri)

  if ray.available_resources().get("CPU", 0) < num_workers+1:
    # If there are no more CPUs left, then shutdown the Serve replicas so we can continue training on the next task.
    serve.shutdown()

  
serve.shutdown()

2022-07-20 21:23:15,792	WARNING read_api.py:291 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
(_get_read_tasks pid=36810) 2022-07-20 21:23:15,789	WARNING torch_datasource.py:56 -- `SimpleTorchDatasource` doesn't support parallel reads. The `parallelism` argument will be ignored.
Read->Map_Batches: 100%|██████████| 1/1 [00:07<00:00,  7.10s/it]
2022-07-20 21:23:29,813	WARNING read_api.py:291 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
(_get_read_tasks pid=36810) 2022-07-20 21:23:29,811	WARNING torch_datasource.py:56 -- `SimpleTorchDatasource` doesn't support parallel reads. The `parallelism` argument will be ignored.
Map P

Starting training for task: 0


Trial name,status,loc,iter,total time (s),loss,_timestamp,_time_this_iter_s
TorchTrainer_e2f7a_00000,TERMINATED,127.0.0.1:36887,4,36.469,820.125,1658377454,6.31464


2022-07-20 21:23:35,495	INFO plugin_schema_manager.py:52 -- Loading the default runtime env schemas: ['/Users/jiaodong/Workspace/ray/python/ray/_private/runtime_env/../../runtime_env/schemas/working_dir_schema.json', '/Users/jiaodong/Workspace/ray/python/ray/_private/runtime_env/../../runtime_env/schemas/pip_schema.json'].
(TorchTrainer pid=36887) 2022-07-20 21:23:38,548	WARNING base_trainer.py:167 -- When passing `datasets` to a Trainer, it is recommended to reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer `scaling_config`. Not doing so can lead to resource contention or hangs. See https://docs.ray.io/en/master/data/key-concepts.html#example-datasets-in-tune for more info.
(_map_block_nosplit pid=36896) /Users/jiaodong/anaconda3/envs/ray3.7/lib/python3.7/site-packages/torchvision/transforms/functional.py:150: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This me

(BaseWorkerMixin pid=36903) loss: 2.361676, epoch: 0, iteration: 0
(BaseWorkerMixin pid=36903) loss: 1.813834, epoch: 0, iteration: 500
(BaseWorkerMixin pid=36903) loss: 1.215798, epoch: 0, iteration: 1000
(BaseWorkerMixin pid=36903) loss: 0.784447, epoch: 0, iteration: 1500
Result for TorchTrainer_e2f7a_00000:
  _time_this_iter_s: 6.658920049667358
  _timestamp: 1658377435
  _training_iteration: 1
  date: 2022-07-20_21-23-56
  done: false
  experiment_id: 19d3571dcd7f4d2b9193be27d191305c
  hostname: Jiaos-MacBook-Pro-16-inch-2019
  iterations_since_restore: 1
  loss: 2443.240612268448
  node_ip: 127.0.0.1
  pid: 36887
  should_checkpoint: true
  time_since_restore: 17.467722177505493
  time_this_iter_s: 17.467722177505493
  time_total_s: 17.467722177505493
  timestamp: 1658377436
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: e2f7a_00000
  warmup_time: 0.003819704055786133
  
(BaseWorkerMixin pid=36903) loss: 0.788217, epoch: 1, iteration: 0
(BaseWorkerMixin pid=3690

...

(BaseWorkerMixin pid=37171) 2022-07-20 21:25:40,627	INFO train_loop_utils.py:298 -- Moving model to device: cpu
(BaseWorkerMixin pid=37171) /Users/jiaodong/Workspace/ray/python/ray/air/_internal/torch_utils.py:64: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:178.)
(BaseWorkerMixin pid=37171)   return torch.as_tensor(vals, dtype=dtype)


(BaseWorkerMixin pid=37171) loss: 1.271604, epoch: 0, iteration: 500
(BaseWorkerMixin pid=37171) loss: 0.773141, epoch: 0, iteration: 1000
(BaseWorkerMixin pid=37171) loss: 0.778621, epoch: 0, iteration: 1500
Result for TorchTrainer_2722b_00000:
  _time_this_iter_s: 6.677475929260254
  _timestamp: 1658377547
  _training_iteration: 1
  date: 2022-07-20_21-25-47
  done: false
  experiment_id: 16c67c09690144f791d7400292b4ecea
  hostname: Jiaos-MacBook-Pro-16-inch-2019
  iterations_since_restore: 1
  loss: 1716.6645558029413
  node_ip: 127.0.0.1
  pid: 37163
  should_checkpoint: true
  time_since_restore: 14.479393005371094
  time_this_iter_s: 14.479393005371094
  time_total_s: 14.479393005371094
  timestamp: 1658377547
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 2722b_00000
  warmup_time: 0.004639148712158203
  
(BaseWorkerMixin pid=37171) loss: 0.640197, epoch: 1, iteration: 0
(BaseWorkerMixin pid=37171) loss: 0.708120, epoch: 1, iteration: 500
(BaseWorkerMixin pid=3

2022-07-20 21:26:07,750	INFO tune.py:738 -- Total run time: 38.07 seconds (37.91 seconds for the tuning loop).
Map Progress (1 actors 1 pending):   0%|          | 0/3 [00:01<?, ?it/s](BlockWorker pid=37237) /Users/jiaodong/anaconda3/envs/ray3.7/lib/python3.7/site-packages/torchvision/transforms/functional.py:150: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:178.)
(BlockWorker pid=37237)   img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()
(BlockWorker pid=37237) /Users/jiaodong/anaconda3/envs/ray3.7/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 

Accuracy for task 3: 0.34796666666666665


Map Progress (1 actors 0 pending): 100%|██████████| 1/1 [00:01<00:00,  1.15s/it]
(ServeController pid=37002) INFO 2022-07-20 21:26:14,682 controller 37002 deployment_state.py:1240 - Stopping 1 replicas of deployment 'mnist_model' with outdated versions.
(ServeController pid=37002) INFO 2022-07-20 21:26:16,797 controller 37002 deployment_state.py:1281 - Adding 1 replicas to deployment 'mnist_model'.
(ServeController pid=37002) INFO 2022-07-20 21:26:19,116 controller 37002 deployment_state.py:1240 - Stopping 1 replicas of deployment 'mnist_model' with outdated versions.
(ServeController pid=37002) INFO 2022-07-20 21:26:21,253 controller 37002 deployment_state.py:1281 - Adding 1 replicas to deployment 'mnist_model'.
(ServeReplica:mnist_model pid=37275) INFO 2022-07-20 21:26:23,715 mnist_model mnist_model#WyGcnk replica.py:483 - HANDLE __call__ OK 0.3ms
(HTTPProxyActor pid=37005) INFO 2022-07-20 21:26:23,716 http_proxy 127.0.0.1 http_proxy.py:320 - POST /mnist_predict 307 4.6ms
(HTTPProxyA

Now that we have finished all of our training, let's see the accuracy of our model after training on each task. 

We should see the accuracy decrease over time. This is to be expected since we are using just a naive fine-tuning strategy so our model is prone to catastrophic forgetting.

As we increase the number of tasks, the model performance on all the tasks trained on so far should decrease.

In [11]:
accuracy_for_all_tasks

[0.3781, 0.37085, 0.34796666666666665]

# [Optional] Step 6: Compare against full training.

We have now incrementally trained our simple multi-layer perceptron. Let's compare the incrementally trained model via fine tuning against a model that is trained on all the tasks up front.

Since we are using a naive fine-tuning strategy, we should expect that our incrementally trained model will perform worse than the the one that is fully trained! However, there's various other strategies that have been developed and are actively being researched to improve accuracy for incremental training. And overall, incremental/continual learning allows you to train in many real world settings where the entire dataset is not available up front, but new data is arriving at a relatively high rate.

Let's first combine all of our datasets for each task into a single, unified Dataset

In [12]:
train_stream = permuted_mnist.generate_train_stream()

# Collect all datasets in the stream into a single dataset.
all_training_datasets = []
for train_dataset in train_stream:
  all_training_datasets.append(train_dataset)
combined_training_dataset = all_training_datasets[0].union(*all_training_datasets[1:])


combined_training_dataset = combined_training_dataset.random_shuffle()

Shuffle Reduce: 100%|██████████| 3/3 [00:01<00:00,  2.60it/s]


Then, we train a new model on the unified Dataset using the same configurations as before.

In [13]:
# Now we do training with the same configurations as before
trainer = TorchTrainer(
            train_loop_per_worker=train_loop_per_worker,
            train_loop_config={
                "num_epochs": num_epochs,
                "learning_rate": learning_rate,
                "momentum": momentum,
                "batch_size": batch_size,
            },
            # Have to specify trainer_resources as 0 so that the example works on Colab. 
            scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu, trainer_resources={"CPU": 0}),
            datasets={"train": combined_training_dataset},
            preprocessor=BatchMapper(fn=preprocess_images),
        )
result = trainer.fit()
full_training_checkpoint = result.checkpoint

2022-07-20 21:26:36,195	WARNING base_trainer.py:167 -- When passing `datasets` to a Trainer, it is recommended to reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer `scaling_config`. Not doing so can lead to resource contention or hangs. See https://docs.ray.io/en/master/data/key-concepts.html#example-datasets-in-tune for more info.


Trial name,status,loc,iter,total time (s),loss,_timestamp,_time_this_iter_s
TorchTrainer_4ec24_00000,TERMINATED,127.0.0.1:37313,4,87.3365,2329.82,1658377686,19.1051


(TorchTrainer pid=37313) 2022-07-20 21:26:39,346	WARNING base_trainer.py:167 -- When passing `datasets` to a Trainer, it is recommended to reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer `scaling_config`. Not doing so can lead to resource contention or hangs. See https://docs.ray.io/en/master/data/key-concepts.html#example-datasets-in-tune for more info.
(_map_block_nosplit pid=37323) /Users/jiaodong/anaconda3/envs/ray3.7/lib/python3.7/site-packages/torchvision/transforms/functional.py:150: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/te

(BaseWorkerMixin pid=37335) loss: 2.340395, epoch: 0, iteration: 0
(BaseWorkerMixin pid=37335) loss: 1.700694, epoch: 0, iteration: 500
(BaseWorkerMixin pid=37335) loss: 1.428385, epoch: 0, iteration: 1000
(BaseWorkerMixin pid=37335) loss: 1.446490, epoch: 0, iteration: 1500
(BaseWorkerMixin pid=37335) loss: 1.130699, epoch: 0, iteration: 2000
(BaseWorkerMixin pid=37335) loss: 1.275075, epoch: 0, iteration: 2500
(BaseWorkerMixin pid=37335) loss: 1.094344, epoch: 0, iteration: 3000
(BaseWorkerMixin pid=37335) loss: 1.151832, epoch: 0, iteration: 3500
(BaseWorkerMixin pid=37335) loss: 0.763781, epoch: 0, iteration: 4000
(BaseWorkerMixin pid=37335) loss: 0.623614, epoch: 0, iteration: 4500
(BaseWorkerMixin pid=37335) loss: 0.584415, epoch: 0, iteration: 5000
(BaseWorkerMixin pid=37335) loss: 0.908524, epoch: 0, iteration: 5500
Result for TorchTrainer_4ec24_00000:
  _time_this_iter_s: 19.464027881622314
  _timestamp: 1658377629
  _training_iteration: 1
  date: 2022-07-20_21-27-09
  done: f

2022-07-20 21:28:07,659	INFO tune.py:738 -- Total run time: 91.46 seconds (91.34 seconds for the tuning loop).


Then, let's test model that was trained on all the tasks up front.

In [14]:
# Then, we used the fully trained model and do batch prediction on the entire test set.

# `full_test_dataset` should already contain the combined test datasets.
fully_trained_accuracy = batch_predict(full_training_checkpoint, full_test_dataset)

Map Progress (1 actors 1 pending):   0%|          | 0/3 [00:01<?, ?it/s](BlockWorker pid=37510) /Users/jiaodong/anaconda3/envs/ray3.7/lib/python3.7/site-packages/torchvision/transforms/functional.py:150: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:178.)
(BlockWorker pid=37510)   img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()
(BlockWorker pid=37510) /Users/jiaodong/anaconda3/envs/ray3.7/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
(BlockWorker pid=37510) A value is trying to be set on a copy of a slice from a DataFrame.
(BlockWorker pid=37

Finally, let's compare the accuracies between the incrementally trained model and the fully trained model. We should see that the fully trained model performs better.

In [15]:
print("Fully trained model accuracy: ", fully_trained_accuracy)
print("Incrementally trained model accuracy: ", accuracy_for_all_tasks[-1])

Fully trained model accuracy:  0.37953333333333333
Incrementally trained model accuracy:  0.34796666666666665


# Next Steps

Once you've completed this notebook, you should be set to play around with scalable incremental training using Ray, either by trying more fancy algorithms for incremental learning other than naive fine-tuning, or attempting to scale out to larger datasets!

If you run into any issues, or have any feature requests, please file an issue on the [Ray Github](https://github.com/ray-project/ray/issues).


